# Razonamiento de los Agentes
Este código muestra cómo implementar razonamiento en agentes que interactúan con el entorno.

## 1.- Setup inicial

### 1.1- Instalar librerías

In [ ]:
#! pip install openai
#! pip install tenacity
#! pip install python-dotenv

### 1.2.- Cargar librerías

In [ ]:
import openai
import os
from dotenv import load_dotenv
from llm.openai import generate_text

### 1.3.- Variables de entorno

In [ ]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt35_model))

### 1.4.- Metodo de utilidad

In [ ]:
def add_message(messages=[], role="user", input=''):
    messages.append({"role": role, "content": input})

### 2.1 - Ejemplo sin razonamiento 

In [ ]:
# Se ejecuta 1 peticion
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas."
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 1: {response}\n") #Resp: ye

# Se ejecuta 2 peticion
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?."
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"\nRespuesta Ejemplo 2: {response}\n") #Resp: 10 Horas

# Se ejecuta 3 peticion
user_input = f"Jane nació el último día de febrero de 2001. Hoy es su cumpleaños de 16 años. Cual es la fecha ayer en MM/DD/YYYY?"
response = generate_text(prompt=user_input,model=gpt35_model,max_tokens=1000)
print(f"\nRespuesta Ejemplo 3: {response}\n") #Resp: 02/27/2017

### 3.1 - Chain Of Thought (CoT)

In [ ]:
# CoT Prompt
COT_PROMPT = "Piensa con sentido comun, detenida y lógicamente, explicando tu respuesta paso a paso."

# Se ejecuta 1 peticion
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 1: {response}\n") #Resp: ye

# Se ejecuta 2 peticion
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"\nRespuesta Ejemplo 2: {response}\n") #Resp: 10 Horas

# Se ejecuta 3 peticion
user_input = f"Jane nació el último día de febrero de 2001. Hoy es su cumpleaños de 16 años. Cual es la fecha ayer en MM/DD/YYYY?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model,max_tokens=1000)
print(f"\nRespuesta Ejemplo 3: {response}\n") #Resp: 02/27/2017

### 3.2 - Chain Of Thought (CoT) + Few Shot

In [ ]:

COT_PROMPT = "Piensa con sentido comun, detenida y lógicamente, explicando tu respuesta paso a paso."

# Inyeccion de mensajes de ejemplo
messages=[]

#Example 1: Concadenate problem
add_message(messages,"user","Take the last letters of the words in 'Elon Musk' and concatenate them.")
add_message(messages,"assistant"," The last letter of 'Elon' is 'n'. The last letter of 'Musk' is 'k'. Concatenating them is 'nk'. The answer is nk.")
#Example 2:  Date Understanding problem
add_message(messages,"user","The concert was scheduled to be on 06/01/1943, but was delayed by one day to today. What is the date 10 days ago in MM/DD/YYYY?")
add_message(messages,"assistant","One day after 06/01/1943 is 06/02/1943, so today is 06/02/1943. 10 days before today is 05/23/1943. So the answer is 05/23/1943.")
add_message(messages,"user","Jane thought today is 3/11/2002, but today is in fact Mar 12, which is 1 day later. What is the date 24 hours later in MM/DD/YYYY?")
add_message(messages,"assistant","Jane thought today is 3/11/2002, but today is in fact Mar 12, which is 1 day later. What is the date 24 hours later in MM/DD/YYYY?")
add_message(messages,"user","Jane thought today is 3/11/2002, but today is in fact Mar 12, which is 1 day later. What is the date 24 hours later in MM/DD/YYYY?")
add_message(messages,"assistant","Today is 03/12/2002. So the date 24 hours later will be 03/13/2002. So the answer is 03/13/2002.")

# Se ejecuta 1 peticion
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 1: {response}\n") #Resp: ye

# Se ejecuta 2 peticion
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model,max_tokens=1000)
print(f"\nRespuesta Ejemplo 2: {response}\n") #Resp: 10 Horas

# Se ejecuta 3 peticion
user_input = f"Jane nació el último día de febrero de 2001. Hoy es su cumpleaños de 16 años. Cual es la fecha ayer en MM/DD/YYYY?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model,max_tokens=1000)
print(f"\nRespuesta Ejemplo 3: {response}\n") #Resp: 02/27/2017

### 3.3.- CoT como un Tree Of Thought (ToT)

In [ ]:
N_TREE_SOLUTIONS=3

PROMPT_COT_LIKE_TOT = f"""
Simula {N_TREE_SOLUTIONS} expertos lógicos brillantes que responden colaborativamente a una pregunta.
Todos los expertos escribirán un paso de pensamiento logico y luego lo compartirán con el grupo.
Se prosigue generando un nuevo paso de pensamiento logico hasta encontrar un concenso.
Se finaliza respondiendo la pregunta deacuerdo con la conclusion final.
"""

user_input = f"""
Bob está en la sala de estar.
Camina hacia la cocina, llevando una taza.
Pone una pelota en la taza y la lleva al dormitorio.
Le da la vuelta a la taza, boca abajo, y luego camina hacia el jardín.
Deja la taza en el jardín y luego camina hacia el garaje.
¿Donde está la pelota? 
"""
# Respuesta: La pelota está en el dormitorio.

messages=[]
add_message(messages,"system",f"{PROMPT_COT_LIKE_TOT}")

# Se ejecuta 1 peticion
response = generate_text(prompt=user_input,model=gpt35_model,messages=messages,max_tokens=2000)
print(f"{response}")

# Se ejecuta 2 peticion
user_input = f"Toma la última letra de cada palabra en 'Larry Page' y concaténalas. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"Respuesta Ejemplo 1: {response}\n") #Resp: ye

# Se ejecuta 3 peticion
user_input = f"Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model)
print(f"\nRespuesta Ejemplo 2: {response}\n") #Resp: 10 Horas

# Se ejecuta 4 peticion
user_input = f"El primer día de 2019 es martes y hoy es el primer lunes de 2019. ¿Cuál es la fecha de hoy en MM/DD/AAAA?. {COT_PROMPT}"
response = generate_text(prompt=user_input,model=gpt35_model,max_tokens=1000)
print(f"\nRespuesta Ejemplo 3: {response}\n") #Resp:  01/07/2019


### 4.1.- Tree Of Thought (ToT) - Generador y Evaluador

In [ ]:
N_SOLUTIONS=3
STEPS=2

PROMPT_TOT_GENERATOR=f"""
I have a problem related to the user request.
Generate {N_SOLUTIONS} differents logical and common sense experts to response only one thought in one step.
Please considere a variety of factors such as the user request, the context, the environment, etc.
"""

PROMPT_TOT_STATE_EVALUATOR=f"""
For each step solution evaluate the posibility to reach the solution form user request and rank each step solution with confidence score.
Also reply with the most likely solution to get the solution.
"""

user_input = """
Tom dejo 3 calcetines en la lavadora, al finalizar, dejo los 3 calcetines secar, luego de 10 horas los calcetines se secaron completamente. ¿Cuánto tiempo llevara secar 20 calcetines?
"""

response=user_input
for i in range(STEPS):
    print(f"----------------------> Step {i+1}:\n")
    # 1.- Descomposición del pensamiento
    messages=[]
    add_message(messages,"system",f"{PROMPT_TOT_GENERATOR}")
    if i != 0:
        add_message(messages,"user",f"{user_input}")
    response = generate_text(prompt=response,model=gpt35_model,messages=messages,max_tokens=1000)
    add_message(messages,"assistant",f"{response}")
    print(f"----> DESCOMPOSITION:\n{response}\n")

    # 2.- Evaluación de pensamiento
    messages=[]
    add_message(messages,"system",f"{PROMPT_TOT_STATE_EVALUATOR}")
    add_message(messages,"user",f"{user_input}")
    response = generate_text(prompt=response,model=gpt35_model,messages=messages,max_tokens=1000)
    add_message(messages,"assistant",f"{response}")
    print(f"----> EVALUATION:\n{response}\n")


### 4.2.- Tree Of Thought (ToT) 4 Fases

In [ ]:
N_SOLUTIONS=3
STEPS=1

PROMPT_TOT_BRAINSTROM=f"""
I have a problem related to the user request. Could you brainstorm {N_SOLUTIONS} distinct solutions? Please considere a variety of factors such as the user request, the context, the environment, etc.
"""

PROMPT_TOT_EVALUATION=f"""
For each of {N_SOLUTIONS} proposed solutions, calculate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assing a probability of success and a confidence level to each option based on these factors.
"""

PROMPT_TOT_EXPANSION=f"""
For each solution deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.
"""

PROMPT_TOT_DECISION=f"""
Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution.
"""

user_input = """
Tengo un problema para determinar la mejor forma para construir una casa. Que me recomendarías para comenzar?
"""

response=user_input

for i in range(STEPS):

    print(f"----------------------> Step {i+1}:\n")
    # 1.- Generador de Ideas
    messages=[]
    add_message(messages,"system",f"{PROMPT_TOT_BRAINSTROM}")
    response = generate_text(prompt=response,model=gpt35_model,messages=messages,max_tokens=1000)
    print(f"----> BRAINSTROM:\n{response}\n")

    # 2.- Evaluador de pensamiento
    messages=[]
    add_message(messages,"system",f"{PROMPT_TOT_EVALUATION}")
    response = generate_text(prompt=response,model=gpt35_model,messages=messages,max_tokens=1000)
    print(f"----> EVALUATION:\n{response}\n")

    # 3.- Expansor de pensamiento
    messages=[]
    add_message(messages,"system",f"{PROMPT_TOT_EXPANSION}")
    response = generate_text(prompt=response,model=gpt35_model,messages=messages,max_tokens=1000)
    print(f"----> EXPANSION:\n{response}\n")

    # 4.- Decision de pensamiento
    messages=[]
    add_message(messages,"system",f"{PROMPT_TOT_DECISION}")
    response = generate_text(prompt=response,model=gpt35_model,messages=messages,max_tokens=1000)
    print(f"----> DECISION:\n{response}\n")